In [11]:
import numpy as np

import HepRead
import HepTools

from HepTools import Scanner as sc

import matplotlib.pyplot as plt

### Set up the directories


In [2]:
model = 'BLSSM'
work_dir = '/scratch/mjad1g20/HEP/WorkArea/BLSSM_Work'
spheno_dir = '/scratch/mjad1g20/HEP/SPHENO/SPheno-3.3.8'
reference_lhs = '/scratch/mjad1g20/HEP/SPHENO/SPheno-3.3.8/BLSSM/Input_Files/LesHouches.in.BLSSM'
madgraph_dir = '/scratch/mjad1g20/HEP/MG5_aMC_v3_1_1'

---

Calling HepTools.Spheno( ) will initiate the Spheno module with an specific model. HepRead.LesHouches will read the reference LesHouches file in initiate an instance for the LesHouches class, so that we can change the parameters of the models and do scans later.




In [3]:
SPhenoBLSSM = HepTools.Spheno(spheno_dir, work_dir ,model)
lhs  = HepRead.LesHouches(reference_lhs, work_dir, model)

BLSSM model activated.
Reading LesHouches from : /scratch/mjad1g20/HEP/SPHENO/SPheno-3.3.8/BLSSM/Input_Files/LesHouches.in.BLSSM


---

We want to recreate the scans made in : [Di-photon decay of a light Higgs state in the BLSSM](https://arxiv.org/pdf/2012.04952.pdf). We use as a reference LesHouches the HighEnergy input for SPheno. They scan through the following parameters: 

$$
\begin{array}{|c|c|}
\hline \text { Parameter } & \text { Range } \\
\hline M_{0} & 100-1000 \mathrm{GeV} \\
\hline M_{\frac{1}{2}} & 1000-4500 \mathrm{GeV} \\
\hline \tan \beta & 1-60 \\
\hline A_{0} & 1000-4000 \mathrm{GeV} \\
\hline
\end{array}
$$

They keep fixed $m_{Z^{\prime}}=2500 \ \mathrm{ GeV}, \tan \beta^{\prime}=1.15$ and $\mu=\mu^{\prime}=B_{\mu}=B_{\mu^{\prime}}=0$. 

- They scan $\tan \beta$ but we will fixed it at $\tan \beta = 5$


In [4]:
print('Blocks in High Energy LesHouches:')
print('---------------------------------')
print(lhs.block_list)

Blocks in High Energy LesHouches:
---------------------------------
['MODSEL', 'SMINPUTS', 'MINPAR', 'EXTPAR', 'SPHENOINPUT', 'DECAYOPTIONS', 'YXIN', 'YVIN']


In [5]:
lhs.block('MODSEL').show()
lhs.block('SMINPUTS').show()

Block MODSEL   #         
1      1                   #  1/0: High/low scale input
2      1                   # Boundary Condition
6      1                   # Generation Mixing
Block SMINPUTS   # STANDARD MODEL INPUTS
2      1.166370E-05        # G_F,Fermi constant
3      1.187000E-01        # alpha_s(MZ) SM MSbar
4      9.118870E+01        # Z-boson pole mass
5      4.180000E+00        # m_b(mb) SM MSbar
6      1.735000E+02        # m_top(pole)
7      1.776690E+00        # m_tau(pole)


In [6]:
lhs.block('MINPAR').set(8,2500)
lhs.block('MINPAR').set(7,1.15)
lhs.block('MINPAR').set(3,5)
lhs.block('MINPAR').show()

# MZp setted to : 2.500000E+03
# TBetaP setted to : 1.150000E+00
# TanBeta setted to : 5.000000E+00
Block MINPAR   # INPUT PARAMETERS
1      1.0000000E+03       # m0         
2      1.5000000E+03       # m12        
3      5.000000E+00        # TanBeta    
4      1.0000000E+00       # SignumMu   
5      -1.5000000E+03      # Azero      
6      1.0000000E+00       # SignumMuP  
7      1.150000E+00        # TBetaP     
8      2.500000E+03        # MZp        


In [7]:
[lhs.block('extpar').set(i,0) for i in [11,12,13,14]]
lhs.block('extpar').set(3,2500)
lhs.block('extpar').set(10,1.15)
lhs.block('extpar').set(1,0.55)
lhs.block('extpar').set(2,-0.12)
lhs.block('extpar').show()

# MuInput setted to : 0.000000E+00
# MuPInput setted to : 0.000000E+00
# BMuInput setted to : 0.000000E+00
# BMuPInput setted to : 0.000000E+00
# MZp setted to : 2.500000E+03
# TBetaP setted to : 1.150000E+00
# gBLinput setted to : 5.500000E-01
# g1BLinput setted to : -1.200000E-01
Block EXTPAR   # INPUT PARAMETERS
1      5.500000E-01        # gBLinput   
2      -1.200000E-01       # g1BLinput  
3      2.500000E+03        # MZp        
10     1.150000E+00        # TBetaP     
11     0.000000E+00        # MuInput    
12     0.000000E+00        # MuPInput   
13     0.000000E+00        # BMuInput   
14     0.000000E+00        # BMuPInput  


Set the SPheno options:

In [8]:
option = [13, 16,78,520,67,60] 
value  = [3,0,1,0,1,1]
for i,j in zip(option, value):
    lhs.block('sphenoinput').set(i,j)
lhs.block('sphenoinput').show()


# 3-Body decays: none (0), fermion (1), scalar (2), both (3) setted to : 3
# One-loop decays setted to : 0
# Output for MadGraph (writes also vanishing blocks) setted to : 1
# Write effective Higgs couplings (HiggsBounds blocks): put 0 to use file with MadGraph! setted to : 0
# effective Higgs mass calculation setted to : 1
# Include possible, kinetic mixing setted to : 1
Block SPHENOINPUT   # SPHENO SPECIFIC INPUT
1      -1                  # error level
2      0                   # SPA conventions
7      0                   # Skip 2-loop Higgs corrections
8      3                   # Method used for two-loop calculation
9      1                   # Gaugeless limit used at two-loop
10     0                   # safe-mode used at two-loop
11     1                   # calculate branching ratios
13     3                   # 3-Body decays: none (0), fermion (1), scalar (2), both (3)
14     0                   # Run couplings to scale of decaying particle
12     1.000E-04           # write 

In [9]:
option = [1,2,3,4,5,6,7,8,9,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013] 
value  = np.zeros(len(option),dtype = int)
for i,j in zip(option, value):
    lhs.block('decayoptions').set(i,int(j))
                                  
lhs.block('decayoptions').show()


# Calc 3-Body decays of Glu setted to : 0
# Calc 3-Body decays of Cha setted to : 0
# Calc 3-Body decays of Chi setted to : 0
# Calc 3-Body decays of Sd setted to : 0
# Calc 3-Body decays of Su setted to : 0
# Calc 3-Body decays of Se setted to : 0
# Calc 3-Body decays of SvRe setted to : 0
# Calc 3-Body decays of SvIm setted to : 0
# Calc 3-Body decays of Fv setted to : 0
# Loop Decay of Sd setted to : 0
# Loop Decay of Su setted to : 0
# Loop Decay of Se setted to : 0
# Loop Decay of SvRe setted to : 0
# Loop Decay of SvIm setted to : 0
# Loop Decay of hh setted to : 0
# Loop Decay of Ah setted to : 0
# Loop Decay of Hpm setted to : 0
# Loop Decay of Glu setted to : 0
# Loop Decay of Cha setted to : 0
# Loop Decay of Chi setted to : 0
# Loop Decay of Fu setted to : 0
# Loop Decay of Fv setted to : 0
Block DECAYOPTIONS   # OPTIONS TO TURN ON/OFF SPECIFIC DECAYS
1      0                   # Calc 3-Body decays of Glu
2      0                   # Calc 3-Body decays of Cha
3      0       

---

So the parameters that we want to scan are (1,$M_{0}$), (2,$M_{\frac{1}{2}}$) and (5,$A_{0}$) wich are in the 'MINPAR' block on the LesHouches file.

In [38]:
m0 = lambda low, high: sc.runiform_float(low,high)
m12 = lambda low, high: sc.runiform_float(low,high)
a0 = lambda low, high: sc.runiform_float(low,high)
tanbeta = lambda low, high: sc.runiform_float(low,high)
N_POINTS = 2

for i in range(N_POINTS):
    lhs.block('MINPAR').set(1,m0(100,1000))
    lhs.block('MINPAR').set(2,m12(1000,4500))
    lhs.block('MINPAR').set(5,a0(1000,4000))
    #lhs.block('MINPAR').set(3,tanbeta(1,60))
    new_lhs_name = 'LesHouches.in.BLSSM_HEscan_'+str(i)
    out_spheno_name = 'SPheno.spc.BLSSM_HEscan_'+str(i)
    # Creating the new LesHouches file, and runing Spheno.
    lhs.new_file(new_lhs_name)
    param_card_path = SPhenoBLSSM.run(new_lhs_name,out_spheno_name)
    # Reading spheno output
    if not(param_card_path==None):
        print('Successful point: ')
        slha = HepRead.Slha(param_card_path)
        print('H1 mass: ',slha.particle(25).mass)
        print('H2 mass: ',slha.particle(35).mass)
    else:
        print('Wrong point')
        

# m0 setted to : 6.369479E+02
# m12 setted to : 1.528573E+03
# Azero setted to : 3.458503E+03
Writing new LesHouches in :/scratch/mjad1g20/HEP/WorkArea/BLSSM_Work/SPhenoBLSSM_input/LesHouches.in.BLSSM_HEscan_0
Save SPheno.spc.BLSSM_HEscan_0 in :/scratch/mjad1g20/HEP/WorkArea/BLSSM_Work/SPhenoBLSSM_output/SPheno.spc.BLSSM_HEscan_0
 Calculating mass spectrum
              1 .-iteration
   ... reached precision:                  Infinity
              2 .-iteration
   ... reached precision:   4.2240859922164510E-002
              3 .-iteration
   ... reached precision:   3.5579881820811511E-003
              4 .-iteration
   ... reached precision:   2.8115761169634232E-004
              5 .-iteration
   ... reached precision:   2.6881926811618546E-005
 Calculate loop corrected masses 
 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!

Successful point: 
H1 mass:  102.296339
H2 mass:  118.985977
# m0 setted to : 9.164002E+02